In [1]:
import sys
sys.path.append('../')

In [2]:
import h5py
from tenpy.tools import hdf5_io
import tenpy
import tenpy.linalg.np_conserved as npc

import os

In [3]:
import numpy as np

from scipy.stats import unitary_group

In [4]:
from SPTOptimization.gradients import *
from SPTOptimization.SymmetryActionWithBoundaryUnitaries import SymmetryActionWithBoundaryUnitaries
from SPTOptimization.utils import (
    tenpy_to_np_transfer_matrix,
    get_left_identity_environment,
    get_right_identity_environment
)

# Load data

In [5]:
DATA_DIR = r"../data/transverse_cluster_200_site_dmrg/0_50.h5"

In [6]:
with h5py.File(DATA_DIR, 'r') as f:
    data = hdf5_io.load_from_hdf5(f)

In [7]:
psi = data['wavefunction']

# Definitions

In [8]:
unitary_sampler = unitary_group(2)

In [9]:
unitary_sampler.rvs()

array([[-0.04779384+0.06430586j, -0.99292934-0.08758905j],
       [ 0.89230815-0.44425969j, -0.06798205-0.04240206j]])

In [10]:
np_I = np.array([[1,0],[0,1]])
np_X = np.array([[0,1],[1,0]])
np_Y = np.array([[0,-1j],[1j,0]])
np_Z = np.array([[1,0],[0,-1]])

In [11]:
test = SymmetryActionWithBoundaryUnitaries(
    psi,
    [np_X, np_I]*50,
    left_boundary_unitaries = [np_Z,],
    right_boundary_unitaries = [np_X, np_Z]
)

In [12]:
test.compute_expectation()

array(0.93061628)

In [13]:
test.compute_svd_approximate_expectation()

(0.9306162804445224+0j)

# Tests

In [14]:
test.right_transfer_matrices

[<npc.Array shape=(8, 8, 8, 8) labels=['vL', 'vR', 'vL*', 'vR*']>,
 <npc.Array shape=(8, 8, 8, 8) labels=['vL', 'vR', 'vL*', 'vR*']>]

In [15]:
test.right_projected_symmetry_state

<npc.Array shape=(8, 8) labels=['vR', 'vR*']>

## identity_environment_functions

In [16]:
ir = get_right_identity_environment(psi, 100)

In [17]:
ir

<npc.Array shape=(8, 8) labels=['vL', 'vL*']>

In [18]:
ir.to_ndarray()

array([[1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1]])

In [19]:
il = get_left_identity_environment(psi, 100)

In [20]:
il

<npc.Array shape=(8, 8) labels=['vR', 'vR*']>

In [21]:
il.to_ndarray()

array([[1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1]])

## expectation_gradient_from_environments

In [22]:
expectation_gradient_from_environments(psi, 100)

<npc.Array shape=(2, 2) labels=['p', 'p*']>

In [23]:
expectation_gradient_from_environments(psi, 100, left_environment=il)

<npc.Array shape=(2, 2) labels=['p*', 'p']>

In [24]:
expectation_gradient_from_environments(psi, 100, right_environment=ir)

<npc.Array shape=(2, 2) labels=['p', 'p*']>

In [25]:
expectation_gradient_from_environments(psi, 100, left_environment=il, right_environment=ir)

<npc.Array shape=(2, 2) labels=['p*', 'p']>

## expectation gradients

In [26]:
psi.get_B(100)

<npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>

In [27]:
a = psi.get_B(100)

In [28]:
a

<npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>

In [29]:
a.legs

[LegCharge(ChargeInfo([], []), qconj=+1,
 array([0, 8]), array([], shape=(1, 0), dtype=int64)),
 LegCharge(ChargeInfo([], []), qconj=+1,
 array([0, 2]), array([], shape=(1, 0), dtype=int64)),
 LegCharge(ChargeInfo([], []), qconj=-1,
 array([0, 8]), array([], shape=(1, 0), dtype=int64))]

In [30]:
right_gradients = expectation_gradients(
    psi,
    test.right_transfer_matrices,
    test.right_symmetry_index+1,
    test.right_projected_symmetry_state
)

In [31]:
len(test.right_transfer_matrices)

2

In [32]:
len(right_gradients)

2

In [33]:
right_gradients[0]

<npc.Array shape=(2, 2) labels=['p*', 'p']>

In [34]:
right_gradients[0].to_ndarray()

array([[0.17654904+0.j, 0.68257359+0.j],
       [0.68257359+0.j, 0.17654904+0.j]])

In [35]:
right_gradients[0].shape

(2, 2)

In [36]:
right_gradients[1]

<npc.Array shape=(2, 2) labels=['p*', 'p']>

In [37]:
right_gradients[1].to_ndarray()

array([[ 6.82573626e-01+0.j,  1.05236871e-08+0.j],
       [ 1.05236870e-08+0.j, -6.82573545e-01+0.j]])

In [38]:
right_gradients[1].shape

(2, 2)

In [39]:
# Careful with np/npc types...
#first_order_correction_from_gradient_one_site(right_gradients[0], np_Z)

In [40]:
gradient = (
    test.left_expectation
    *test.symmetry_transfer_matrix_singular_vals[0]
    *right_gradients[0].to_ndarray()
)
delta_U = np_X

In [41]:
np.sum(gradient*delta_U)

(0.9306162804445222+0j)

In [42]:
test2 = SymmetryActionWithBoundaryUnitaries(
    psi,
    [np_X, np_I]*50,
    left_boundary_unitaries = [np_Z,],
    right_boundary_unitaries = [np_X + np_X, np_Z]
)

In [43]:
test2.compute_expectation()

array(1.86123256)

In [44]:
test2.expectation - test.expectation

0.9306162804445213

## linear_expectation_gradient

Not much to check here

In [45]:
linear_expectation_gradient(gradient, test.expectation)

array([[0.12035289+0.j, 0.46530814+0.j],
       [0.46530814+0.j, 0.12035289+0.j]])

In [46]:
gradient

array([[0.12035289+0.j, 0.46530814+0.j],
       [0.46530814+0.j, 0.12035289+0.j]])

## gradients_outside_right_boundary_unitaries

In [68]:
outside_gradients = gradients_outside_right_boundary_unitaries(
    psi,
    (
        test.left_expectation
        * test.symmetry_transfer_matrix_singular_vals[0]
        * test.right_transfer_vectors[-1]
    ),
    3,
    test.right_symmetry_index + 2
)

In [69]:
outside_gradients

[<npc.Array shape=(2, 2) labels=['p*', 'p']>,
 <npc.Array shape=(2, 2) labels=['p*', 'p']>,
 <npc.Array shape=(2, 2) labels=['p*', 'p']>]

In [70]:
outside_gradients[0].to_ndarray()

array([[0.46530814+0.j, 0.12035289+0.j],
       [0.12035289+0.j, 0.46530814+0.j]])

In [71]:
test3 = SymmetryActionWithBoundaryUnitaries(
    psi,
    [np_X, np_I]*50,
    left_boundary_unitaries = [np_Z,],
    right_boundary_unitaries = [np_X, np_Z, np_X]
)

In [72]:
test3.compute_expectation()

array(0.24070578)

As expected.

# Old code

In [49]:
x = 7

In [50]:
isinstance(x, int)

True

In [51]:
isinstance('a', int)

False

In [52]:
help(range)

Help on class range in module builtins:

class range(object)
 |  range(stop) -> range object
 |  range(start, stop[, step]) -> range object
 |  
 |  Return an object that produces a sequence of integers from start (inclusive)
 |  to stop (exclusive) by step.  range(i, j) produces i, i+1, i+2, ..., j-1.
 |  start defaults to 0, and stop is omitted!  range(4) produces 0, 1, 2, 3.
 |  These are exactly the valid indices for a list of 4 elements.
 |  When step is given, it specifies the increment (or decrement).
 |  
 |  Methods defined here:
 |  
 |  __bool__(self, /)
 |      True if self else False
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(self, key, /)
 |      Return self[key].
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __hash

In [53]:
kd_00 = np.array([[1,0], [0, 0]])
kd_01 = np.array([[0,1], [0, 0]])
kd_10 = np.array([[0,0], [1, 0]])
kd_11 = np.array([[0,0], [0, 1]])

In [54]:
kd_arrays = [kd_00, kd_01, kd_10, kd_11]

In [55]:
kd_tests = [
    SymmetryActionWithBoundaryUnitaries(
        psi,
        [np_X, np_I]*50,
        left_boundary_unitaries = [np_Z,],
        right_boundary_unitaries = [m, np_Z]
    )
    for m in kd_arrays
]

In [56]:
for t in kd_tests:
    t.compute_expectation()

In [57]:
kd_expectations = [t.expectation for t in kd_tests]

In [58]:
kd_expectations

[array(0.12035289), array(0.46530814), array(0.46530814), array(0.12035289)]

In [59]:
gradient

array([[0.12035289+0.j, 0.46530814+0.j],
       [0.46530814+0.j, 0.12035289+0.j]])

In [60]:
0.17654904/0.12035289

1.466928131098472

In [61]:
0.68257359/0.46530814

1.4669281091880317

In [62]:
(0.68257359/0.46530814)**(-1)

0.6816966651170902

In [63]:
test.symmetry_transfer_matrix_singular_vals

array([4.99357640e-01, 5.64338811e-17, 3.52817003e-17, 3.52817003e-17,
       3.52817003e-17, 3.52817003e-17, 3.52817003e-17, 3.52817003e-17,
       3.52817003e-17, 3.52817003e-17, 3.52817003e-17, 3.52817003e-17,
       3.52817003e-17, 3.52817003e-17, 3.52817003e-17, 3.52817003e-17,
       3.52817003e-17, 3.52817003e-17, 3.52817003e-17, 3.52817003e-17,
       3.52817003e-17, 3.52817003e-17, 3.52817003e-17, 3.52817003e-17,
       3.52817003e-17, 3.52817003e-17, 3.52817003e-17, 3.52817003e-17,
       3.52817003e-17, 3.52817003e-17, 3.52817003e-17, 3.52817003e-17,
       3.52817003e-17, 3.52817003e-17, 3.52817003e-17, 3.52817003e-17,
       3.52817003e-17, 3.52817003e-17, 3.52817003e-17, 3.52817003e-17,
       3.52817003e-17, 3.52817003e-17, 3.52817003e-17, 3.52817003e-17,
       3.52817003e-17, 3.52817003e-17, 3.52817003e-17, 3.52817003e-17,
       3.52817003e-17, 3.52817003e-17, 3.52817003e-17, 3.52817003e-17,
       3.52817003e-17, 3.52817003e-17, 3.52817003e-17, 3.52817003e-17,
      

In [64]:
test.left_expectation

(1.3651471712007195+0j)

In [65]:
test.left_expectation*(test.symmetry_transfer_matrix_singular_vals[0])

(0.6816966698367003+0j)